In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimpy import skim
from skimpy import clean_columns


from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier

In [ ]:
# df.outcome.value_counts()

outcome
0    500
1    268
Name: count, dtype: int64

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimpy import skim
from skimpy import clean_columns


from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
from catboost import CatBoostClassifier

# Загрузка и подготовка данных
df = pd.read_csv('../data/diabetes.csv')
df = clean_columns(df)  # Ваша функция очистки

# Разделение данных
X = df.drop('outcome', axis=1)
y = df['outcome']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    stratify=y, 
    test_size=0.2, 
    random_state=42
)

# Инициализация и обучение CatBoost
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    eval_metric='Recall',
    random_state=42,
    verbose=100
)

model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    early_stopping_rounds=50
)

# Предсказания и оценка
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("\n=== Метрики модели ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}\n")

0:	learn: 0.5560748	test: 0.5185185	best: 0.5185185 (0)	total: 73.5ms	remaining: 36.7s
100:	learn: 0.9112150	test: 0.6111111	best: 0.6111111 (68)	total: 202ms	remaining: 799ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6111111111
bestIteration = 68

Shrink model to first 69 iterations.

=== Метрики модели ===
Accuracy: 0.7403
Recall: 0.6111
Precision: 0.6346
F1-score: 0.6226
ROC-AUC: 0.8091



In [ ]:
# # Первые 5 строк
# print(df.head())

# # Информация о данных
# print(df.info())

# # Статистика
# print(df.describe())

# # Проверка на пропуски
# print(df.isnull().sum())  

In [13]:
# plt.figure(figsize=(12, 8))
# for i, column in enumerate(df.columns[:-1]):
#     plt.subplot(3, 3, i+1)
#     sns.histplot(df[column], kde=True, bins=20)
#     plt.title(column)
# plt.tight_layout()
# plt.show()

In [14]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# sns.boxplot(x='outcome', y='glucose', data=df)
# plt.title("Уровень глюкозы у пациентов с диабетом и без")
# plt.show()

In [15]:
# plt.figure(figsize=(10, 6))
# sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
# plt.title('Корреляционная матрица')
# plt.show()

Точность модели: 0.62


In [186]:
import pandas as pd
import numpy as np

# Загрузка данных

# 1. Взаимодействие признаков
df['bmi_glucose_interaction'] = df['bmi'] * df['glucose'] / 100  # Нормировка для удобства
df['bloodpressure_age_ratio'] = df['blood_pressure'] / df['age']

# 2. Категоризация возраста
df['age_group'] = pd.cut(df['age'], bins=[0, 30, 50, 100], labels=['low', 'middle', 'hight'])

# Проверка результата
print("Новые признаки:")
display(df[['bmi_glucose_interaction', 'bloodpressure_age_ratio', 'age_group']].head())

Новые признаки:


,bmi_glucose_interaction,bloodpressure_age_ratio,age_group
0,49.728,1.440000,middle
1,22.610,2.129032,middle
2,42.639,2.000000,middle
3,25.009,3.142857,low
4,59.047,1.212121,middle


In [16]:
# # Тепловая карта корреляций
# plt.figure(figsize=(12, 8))
# corr = df.corr(numeric_only=True)
# sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title("Корреляция признаков (включая новые)")
# plt.show()

# # Топ-3 коррелирующих признака
# top_corr = corr['outcome'].abs().sort_values(ascending=False).index[1:4]
# print(f"Топ-3 признака, связанных с диабетом: {list(top_corr)}")

In [ ]:
# 3. Проверка гипотезы
# Гипотеза:
# Комбинация высокого уровня глюкозы (Glucose > 140) и высокого ИМТ (BMI > 30) увеличивает риск диабета в 2+ раза.

In [190]:
# Создание бинарных признаков
df['high_glucose'] = (df['glucose'] > 140).astype(int)
df['high_bmi'] = (df['bmi'] > 30).astype(int)
df['high_risk_group'] = (df['high_glucose'] + df['high_bmi'] >= 2).astype(int)

# Проверка гипотезы
risk_group_rate = df[df['high_risk_group'] == 1]['outcome'].mean()
non_risk_group_rate = df[df['high_risk_group'] == 0]['outcome'].mean()

print(f"Риск диабета в группе High_Risk: {risk_group_rate:.2%}")
print(f"Риск диабета в остальных случаях: {non_risk_group_rate:.2%}")
print(f"Относительный риск: {risk_group_rate / non_risk_group_rate:.1f}x")

Риск диабета в группе High_Risk: 75.68%
Риск диабета в остальных случаях: 25.16%
Относительный риск: 3.0x


In [21]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, recall_score, 
                            precision_score, roc_auc_score, f1_score)
from catboost import CatBoostClassifier
import json
import joblib
from datetime import datetime
from skimpy import skim
from skimpy import clean_columns

# Настройка путей
BASE_DIR = Path(__file__).parent.parent
DATA_PATH = BASE_DIR / 'data' / 'diabetes.csv'
MODEL_DIR = BASE_DIR / 'models'
os.makedirs(MODEL_DIR, exist_ok=True)

def train_and_save_model():
    # Загрузка и подготовка данных
    print("Загрузка данных...")
    df = pd.read_csv(DATA_PATH)
    df = clean_columns(df)
    
    # Разделение данных
    X = df.drop('outcome', axis=1)
    y = df['outcome']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        stratify=y, 
        test_size=0.2, 
        random_state=42
    )
    
    # Обучение модели
    print("Обучение CatBoost модели...")
    model = CatBoostClassifier(
        iterations=500,
        learning_rate=0.1,
        depth=6,
        eval_metric='Recall',
        random_seed=42,
        verbose=100
    )
    
    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=50
    )
    
    # Оценка модели
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_proba),
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Сохранение модели и метрик
    model_version = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_name = f"catboost_diabetes_{model_version}"
    
    print(f" Сохранение модели {model_name}...")
    joblib.dump(model, MODEL_DIR / f"{model_name}.joblib")
    
    with open(MODEL_DIR / f"{model_name}_metrics.json", 'w') as f:
        json.dump(metrics, f, indent=4)
    
    print("\nОбучение завершено!")
    print("Метрики модели:")
    for k, v in metrics.items():
        if k == 'timestamp':
            print(f"{k.upper()}: {v}")
        else:
            print(f"{k.upper()}: {v:.4f}")
    
    return model, metrics

if __name__ == "__main__":
    train_and_save_model()

NameError: name '__file__' is not defined